In [1]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json

ITEM_COUNTS = 100

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'Display-Api-Key': 'VWmkUPgs6g2fviPZ5JQFQ3pERP4tIXv/J2jppLqSRBk='
}

In [3]:
middle_category_nums = ['10101201', '10101202', '10101203', '10101204', '10101205',
                        '10101206', '10101207', '10101208', '10101209', '10101210',
                        '10101211', '10101212']

In [24]:
def get_item_cds(middle_category_num, gender):
    url = 'https://api-display.wconcept.co.kr/display/api/v2/best/products'

    if gender == 'men':
        genderType = 'men'
    else:
        genderType = 'women'

    data = {
    "custNo": "",
    "dateType": "daily",
    "domain": 'WOMEN',
    "genderType": genderType,
    "depth1Code": "10101",
    "depth2Code": middle_category_num,
    "pageNo": 1,
    "pageSize": ITEM_COUNTS
    }

    response = requests.post(url, headers=headers, json=data)

    soup = BeautifulSoup(response.text, 'lxml')
    info = soup.string
    info = json.loads(info)
    info = info['data']['content']

    item_cds = []

    for item in info:
        item_cds.append(item['itemCd'])

    return item_cds

In [25]:
def get_item_data(info):
    # 브랜드
    brandNameKr = info['brandNameKr']

    # 품번
    itemCd = info['itemCd']

    # 좋아요
    heartCnt = info['heartCnt']

    # 품절 여부
    statusName = info['statusName']

    # 정상가
    customerPrice = info['customerPrice']

    # 쿠폰 적용가 (수정 필요할수도)
    finalPrice = info['finalPrice']

    # 색상

    # 사이즈

    # 이미지

    # 상품명
    itemName = info['itemName']

    data = {
        'brandNameKr': brandNameKr,
        'itemCd': itemCd,
        'heartCnt': heartCnt,
        'statusName': statusName,
        'customerPrice': customerPrice,
        'finalPrice': finalPrice,
        'itemName': itemName
    }

    return data



In [26]:
def get_item_info(item_cd):
    
    url = f'https://www.wconcept.co.kr/Ajax/GetProductsInfo'
    data = {'itemcds': item_cd}
    response = requests.post(url, headers=headers, data=data)

    soup = BeautifulSoup(response.text, 'lxml')
    info = json.loads(soup.string)[0]
    
    info = get_item_data(info)

    return info



In [27]:
def main():
    for gender in ['men', 'women']:
        for middle_category_num in middle_category_nums:
            item_cds = get_item_cds(middle_category_num, gender)
            item_info = []
            for item_cd in item_cds:
                info = get_item_info(item_cd)
                item_info.append(info)
                print(info)
    

In [28]:
if __name__ == '__main__':
    main()

{'brandNameKr': '에드워드맥스', 'itemCd': '304205825', 'heartCnt': 6.0, 'statusName': '판매중', 'customerPrice': 349000.0, 'finalPrice': 349000.0, 'itemName': 'E-BATTEN (SINGLE BLAZER) '}
{'brandNameKr': '비르반테', 'itemCd': '302747914', 'heartCnt': 5.0, 'statusName': '판매중', 'customerPrice': 459000.0, 'finalPrice': 321300.0, 'itemName': '피크트 레귤러핏 린넨 자켓 - 네이비'}
{'brandNameKr': '그레이버', 'itemCd': '301203422', 'heartCnt': 1098.0, 'statusName': '판매중', 'customerPrice': 99000.0, 'finalPrice': 69300.0, 'itemName': '[UNISEX]플라워 도트자수 윈드브레이커_아이보리'}
{'brandNameKr': '발루트', 'itemCd': '301515278', 'heartCnt': 1.0, 'statusName': '판매중', 'customerPrice': 158000.0, 'finalPrice': 158000.0, 'itemName': 'MISTY PARKA (NAVY WASHED)'}
{'brandNameKr': '노스페이스', 'itemCd': '305352957', 'heartCnt': 21.0, 'statusName': '판매중', 'customerPrice': 148000.0, 'finalPrice': 133200.0, 'itemName': 'NJ3BQ10L 화이트라벨 올레마 EX 자켓'}
{'brandNameKr': '아르반', 'itemCd': '305736726', 'heartCnt': 0.0, 'statusName': '판매중', 'customerPrice': 158000.0, 'fi